<a href="https://colab.research.google.com/github/michalis0/DataScience_and_MachineLearning/blob/master/Assignements/Part%204/Assignment_part_four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DSML investigation:

You are part of the Suisse Impossible Mission Force, or SIMF for short. You need to uncover a rogue agent that is trying to steal sensitive information.

Your mission, should you choose to accept it, is to find that agent before stealing any classified information. Good luck!

# Assignement part four
### Due 22.10
#### Identifying the suspects credit score
We received informations that the rogue agent has a good credit score.

Our spies at SIMF have managed to collect financial information relating to our suspects as well as a training dataset.

Create a Neural Network over the training dataset `df` to identify which of the suspects have a good Credit_Mix


## Getting to know our data

* Age: a users age
* Occupation: a users employment field
* Annual_Income: a users annual income
* Monthly_Inh_Salary: the calculated salary received by a given user on a monthly basis
* Num_Bank_Accounts: the number of bank accounts possessed by a given user
* Num_Credit_Cards: the number of credit card given user possesses
* Interest_Rate: The interest rate on those cards (if multiple then its the average)
* Num_of_Loans: The number of loans of each user
* Delay_from_due_date: payment tardiness of user
* Num_of_Delayed_Payment: the count of delayed payments
* Changed_Credit_Limit: NaN
* Num_Credit_Inquiries: NaN
* Credit_Mix: The users credit score
* Outsting_Debt: Outstanding debt
* Credit_Utilization_Ratio: the percentage of borrowed money over borrowing allowance
* Payment_of_Min_Amount: does the user usually pay the minimal amount (categorical)
* Total_EMI_per_month: Monthly repayments to be made
* Amount_invested_monthly: The amout put in an investment fun by the user on a monthly basis
* Payment_Behaviour: the users payment behavior (categorical)
* Monthly_Balance: The users end of the month balance
* AutoLoan: If the user has an active loan for their vehicule
* Credit-BuilderLoan: If the user has a loan to increase their credit score
* DebtConsolidationLoan, HomeEquityLoan, MortgageLoan, NotSpecified, PaydayLoan, PersonalLoan, StudentLoan: different types of loans(categorical features)



In [1]:
# Import required packages
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/train_classification.csv", index_col='Unnamed: 0').dropna()
suspects = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/suspects.csv", index_col='Unnamed: 0').dropna()

In [3]:
df.head()

,Age,Occupation,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Monthly_Balance,AutoLoan,Credit-BuilderLoan,DebtConsolidationLoan,HomeEquityLoan,MortgageLoan,NotSpecified,PaydayLoan,PersonalLoan,StudentLoan
0,23,Scientist,19114.12,1824.843333,3,4,3,4,3,7,...,186.266702,1,1,0,1,0,0,0,1,0
1,24,Scientist,19114.12,1824.843333,3,4,3,4,3,9,...,361.444004,1,1,0,1,0,0,0,1,0
3,24,Scientist,19114.12,4182.004291,3,4,3,4,4,5,...,343.826873,1,1,0,1,0,0,0,1,0
5,28,Teacher,34847.84,3037.986667,2,4,6,1,3,3,...,303.355083,0,1,0,0,0,0,0,0,0
8,35,Engineer,143162.64,4182.004291,1,5,8,3,8,1942,...,854.226027,2,0,0,0,0,1,0,0,0


In [4]:
df['Payment_of_Min_Amount'].head()

0    No
1    No
3    No
5    No
8    No
Name: Payment_of_Min_Amount, dtype: object

In [5]:
df["Credit_Mix"].value_counts()

Standard    13421
Good         8963
Bad          6839
Name: Credit_Mix, dtype: int64

# 1. Preparing the data
## 1.1 Data cleaning
 Perform OHE over the "Occupation" feature

 Then, perform LE over Payment_of_Min_Amount and Payment_Behaviour

 _hint: As we will be testing only one model no need to define a pipeline_

In [6]:
# reseting the indexes so it works
df = df.reset_index(drop=True)

# creating the OHE model and applying OneHotEncoding over the 'Occupation' feature
OneHot = OneHotEncoder()
OHE_Occ = OneHot.fit_transform(df[['Occupation']])

# creating the LabelEncoder model
LabEnc = LabelEncoder()

# applying LabelEncoder over Payment_of_Min_Amount and Payment_Behaviour
df['Payment_of_Min_Amount'] = LabEnc.fit_transform(df['Payment_of_Min_Amount'])
df['Payment_Behaviour'] = LabEnc.fit_transform(df['Payment_Behaviour'])

# concatenating the dataset
df = pd.concat([df.drop(columns=['Occupation']), pd.DataFrame(OHE_Occ.toarray(), columns=OneHot.get_feature_names_out(['Occupation']))], axis=1)

Making sure that the new dataset has been modified :

In [7]:
# checking payment of min amount
df['Payment_of_Min_Amount'].head()

0    1
1    1
2    1
3    1
4    1
Name: Payment_of_Min_Amount, dtype: int64

In [8]:
# checking payment_behaviour
df['Payment_Behaviour'].head()

0    6
1    2
2    2
3    4
4    5
Name: Payment_Behaviour, dtype: int64

In [9]:
# checking the entire dataset
df.head()

,Age,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_MediaManager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer
0,23,19114.12,1824.843333,3,4,3,4,3,7,11.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,24,19114.12,1824.843333,3,4,3,4,3,9,13.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,24,19114.12,4182.004291,3,4,3,4,4,5,11.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,28,34847.84,3037.986667,2,4,6,1,3,3,5.42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,35,143162.64,4182.004291,1,5,8,3,8,1942,7.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1.2 Dataset splitting

Split the dataset in two, first X with your independent features and then y with the dependent feature **CreditMix**.

Then perform :
* OneHotEncoding over the **CreditMix** feature.
* A MinMaxScaller over the independent features

In [10]:
# making sure first that all my features have been transformed to a numerical value (beside CreditMix)
#df.dtypes

In [11]:
# features = df.drop(columns=['Credit_Mix'])
target = ['Credit_Mix']

X = df.drop(columns=['Credit_Mix'])
y = df[target]

In [12]:
# OneHotEncoding over the Credit_Mix feature
New_OHE = OneHotEncoder()
OHE_CM = New_OHE.fit_transform(y[['Credit_Mix']])
y = pd.concat([y.drop(columns=['Credit_Mix']), pd.DataFrame(OHE_CM.toarray(), columns=New_OHE.get_feature_names_out(['Credit_Mix']))], axis=1)

# making sure that everything is correct
display(y)

,Credit_Mix_Bad,Credit_Mix_Good,Credit_Mix_Standard
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
29218,0.0,1.0,0.0
29219,1.0,0.0,0.0
29220,1.0,0.0,0.0
29221,0.0,1.0,0.0


In [13]:
#Define the scaler
sc = MinMaxScaler()

#Fit the scaler
X = sc.fit_transform(X)

### 1.2.1 Train Test splitting
Now split the data in X_train, X_test, y_train, y_test,

You can use test_size = 0.2 and a random_state of 42

In [14]:
# train-test-split on X and y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 1.2.2 final touches
Convert your datasets to `Torch tensors` of type `torch.float`

In [15]:
# converting the previous datasets (X_train, X_test, y_train, and y_test) to PyTorch tensors

# Note from the Week5 notebook : A torch.Tensor is a multi-dimensional matrix containing elements of a single data type. #just to remember what the code below is about

X_train = torch.tensor(X_train, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)
y_train = torch.tensor(y_train.values, dtype=torch.float)
y_test = torch.tensor(y_test.values, dtype=torch.float)

# 2 Model preparation:

## 2.1 Define a Neural network model and instantiate it.
You can set the number of neurons to 150.

In [16]:
# defining a neural network class here:
# I used the class that has been shared in class ! (https://github.com/michalis0/DataScience_and_MachineLearning/blob/master/Week_5/Week_5_solutions.ipynb)

class Net(nn.Module):
    def __init__(self, D_in, neurons_nbr, D_out):
        super(Net, self).__init__()

        self.linear1 = nn.Linear(D_in, neurons_nbr)
        self.linear2 = nn.Linear(neurons_nbr, D_out)
        self.activation = nn.ReLU()

    def forward(self, x):
        y_pred = self.activation(self.linear1(x))
        y_pred = self.linear2(y_pred)
        return y_pred

In [17]:
# Instantiate your model here
D_in, D_out = X_train.shape[1], y_train.shape[1]
NN_model = Net(D_in, 150, D_out)

# let's check how many paramaters there are in the model (question 1 of the moodle assignment)
total_parameters = sum(p.numel() for p in NN_model.parameters())
print("Total number of parameters : ", total_parameters)

Total number of parameters :  6903


## 2.2 finding the best model:
Identify, amongst the following options the best parameters for your model:

* `criterion` : [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), [BCEWithLogitsLoss](hhttps://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)
* `iterations` : 150, 250, 500
* `learning rate` : 0.00005, 0.001, 12.031


_Hint: restart your runtime between each execution to ensure that previous neural networks dont interfere with your current one_

_You can evaluate your model based on it's accuracy over the test set_

In [18]:
# putting what it is written in the subject in some variables

loss_criterion1 = nn.CrossEntropyLoss(reduction='sum') # first loss function
loss_criterion2 = nn.BCEWithLogitsLoss(reduction='sum') # second loss function
iterations = [150, 250, 500]
learning_rates = [0.00005, 0.001, 12.031]

# then i will test manually all the elements above :)

In [19]:
# Define your Adam optimizer for finding the weights of the network here
optimizer = torch.optim.Adam(NN_model.parameters(), lr=learning_rates[1])

In [20]:
# here again I used the code that has been explained during the walk through :)

losses = []
losses_test= []

for t in range(iterations[2]):

    # Forward pass: computing prediction on training set
    y_train_pred = NN_model(X_train)

    # Computing loss
    loss = loss_criterion1(y_train_pred, y_train)
    losses.append(loss.item())
    if torch.isnan(loss):
        break

    # Computing gradient
    optimizer.zero_grad()
    loss.backward()

    # Updating
    optimizer.step()

    # Computing loss on test set
    losses_test.append(loss_criterion1(NN_model(X_test), y_test).item())

## 2.3 Model Accuracy
Identify the models accuracy over the train and test parts of the training dataset

In [21]:
# accuracy function

# I used ChatGPT to compute this one function (bc i couldn't find anything on the walk through or on the slides and the normal accuracy function didn't work !):

def calculate_accuracy(model, inputs, targets):
  outputs = model(inputs)
  _, predicted_classes = torch.max(outputs, dim=1)
  true_classes = torch.max(targets, dim=1)[1]
  correct = (predicted_classes == true_classes).sum().item()
  total = targets.size(0)
  return correct / total

In [22]:
# deactivate dropout layers
NN_model.eval() #not sure if it's right

# calculating accuracies
with torch.no_grad():
    acc_train = calculate_accuracy(NN_model, X_train, y_train)
    acc_test = calculate_accuracy(NN_model, X_test, y_test)

    print("Train accuracy :", acc_train)
    print("Test accuracy :" , acc_test)


Train accuracy : 0.8405766104884934
Test accuracy : 0.8331907613344739


In [23]:
# Here are the results for the previous tests :

# CEL :
  # 150 & 0.00005 :   0.45457
  # 150 & 0.001 :     0.76951
  # 150 & 12qqch :    0.45457

  # 250 & 0.00005 :   0.45919
  # 250 & 0.001 :     0.81094
  # 250 & 12qqch :    0.45457

  # 500 & 0.00005 :   0.46991
  # 500 & 0.001 :     0.83934   ## this one win ##
  # 500 & 12qqch :    0.45457

# BCE
  # 150 & 0.00005 :   0.45799
  # 150 & 0.001 :     0.74165
  # 150 & 12qqch :    0.45457

  # 250 & 0.00005 :   0.46894
  # 250 & 0.001 :     0.776903
  # 250 & 12qqch :    0.45457

  # 500 & 0.00005 :   0.45508
  # 500 & 0.001 :     0.82172
  # 500 & 12qqch :    0.45457

  # So the best (those who have the highest accuracy) parameters are :
    # -Loss function  : CEL
    # - Iteration     : 500
    # - LR            : 0.001

# 3. Predictions over the suspects dataset
## 3.1 Retrain a new model over the full training dataset
#### Please use the following parameters for this section:
* ``neurons`` = 150
* ``learning`` rate = 0.00005
* ``criterion`` = CrossEntropyLoss
* `iterations` = 500

_hint you may have to redo some preprocessing as you did in part one_

In [24]:
# defining some variables with the given values

neurons = 150
new_iter = 500;
new_learning_rate = 0.00005
new_criterion = nn.CrossEntropyLoss(reduction='sum')

In [25]:
# redefining with the complete data
target = ['Credit_Mix']

X2 = df.drop(columns='Credit_Mix')
y2 = df[target]

In [26]:
# let's put Credit Mix in a OHE and fit the necessary data :

ohe = OneHotEncoder(sparse_output=False)
y2e = ohe.fit_transform(y2.values.reshape(-1, 1))

# transforming the data
new_sc = MinMaxScaler()
X2 = new_sc.fit_transform(X2)
sus = new_sc.transform(suspects.drop(columns='userID'))

In [27]:
# converting the necessary datasets to PyTorch tensors

X2 = torch.tensor(X2, dtype=torch.float)
y2 = torch.tensor(y2e, dtype=torch.float)
sus = torch.tensor(sus, dtype=torch.float)


In [28]:
# creating a new model :
D_in, D_out = X2.shape[1], y2.shape[1]
new_NN_model = Net(D_in, 150, D_out)

In [29]:
# Define your Adam optimizer for finding the weights of the network here:
new_optimizer = torch.optim.Adam(new_NN_model.parameters(), new_learning_rate)

In [30]:
# perform your training here (i)
for i in range(new_iter):

    # forward pass : computing prediction on the full dataset
    predictions = new_NN_model(X2)

    # computing loss
    loss = new_criterion(predictions, torch.max(y2, 1)[1])

    # computing gradient
    new_optimizer.zero_grad()
    loss.backward()
    # updating
    new_optimizer.step()

## 3.2 Predict over the suspects dataset

In [31]:
# dropout layers
new_NN_model.eval()

# predicting over the suspects dataset
with torch.no_grad():
    pred_sus = torch.argmax(new_NN_model(sus), dim=1).numpy()

suspects['Credit_Mix'] = pred_sus

In [32]:
# searching if one suspect from the question 5 on moodle appears in the remaining suspects :

remaining_suspects = suspects[(suspects["Credit_Mix"] == 0)]

#display the remaining_suspects who have at least the same userID that the ones one moodle :

for userID in [241892, 173906, 539227, 582281, 317991]: # the users that have been given in the moodle
    if userID in remaining_suspects["userID"].values:
        print(userID)

539227
